In [1]:
import torch
import torchvision
import numpy as np
from PIL import Image
import requests
import matplotlib.pyplot as plt
from torchvision import models
from torchvision import transforms

In [2]:
io_mapping = []
with open('IO_places365.txt') as f:
    for line in f:
        parts = line.strip().split()
        io_mapping.append(int(parts[1]))
io_mapping = np.array(io_mapping)

In [3]:
model = models.alexnet(num_classes=365)
checkpoint = torch.hub.load_state_dict_from_url(
    'http://places2.csail.mit.edu/models_places365/alexnet_places365.pth.tar',
    map_location=torch.device('cpu')
)
state_dict = {k.replace('module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
preprocess = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

In [5]:
def classify_indoor_outdoor(img_path):
    img = Image.open(img_path).convert('RGB')
    input_tensor = preprocess(img).unsqueeze(0)

    with torch.no_grad():
        logit = model(input_tensor)
        probs = torch.nn.functional.softmax(logit, dim=1).cpu().numpy()[0]

    io_mapping_binary = (io_mapping == 2).astype(int)  # 0=Indoor, 1=Outdoor
    indoor_prob = np.sum(probs[io_mapping_binary == 0])
    outdoor_prob = np.sum(probs[io_mapping_binary == 1])

    return "Indoor" if indoor_prob > outdoor_prob else "Outdoor"

In [6]:
img_path = 'trial/indoor_2.jpg'  # replace with image_path
result = classify_indoor_outdoor(img_path)
print("Prediction:", result)

Prediction: Indoor


In [7]:
img_path = 'trial/outdoor_2.jpg'
result = classify_indoor_outdoor(img_path)
print("Prediction:", result)

Prediction: Outdoor
